## Multiplication of images

variates images by albumentation to get a higher number of input data for deep learning (modified from ix.de/zuqv).

Creates 45 (5 x 9) new images per original image.

modifications by github.com/th-under

May 2021




In [2]:
import os
import glob
import cv2

from albumentations import CenterCrop, VerticalFlip, HorizontalFlip, \
    ElasticTransform, RandomRotate90, RandomContrast, RandomBrightness, \
    ToGray, ToSepia, Blur, MultiplicativeNoise, JpegCompression, ChannelDropout


In [3]:
INFOLDER = 'my_output/'   # folder containing original images relative to working directory
OUTFOLDER = ''            # directory to write variations to
EXTENSION = '.png'        # image name extension
IMAGE_SIZE = 224          # image dimension in pixels (x and y equal)



raw_images = os.getcwd() + '/' + INFOLDER
processed_images = raw_images + OUTFOLDER

raw_image_lst = glob.glob(raw_images + '*' + EXTENSION)


In [4]:
print(f"{len(raw_image_lst)} images found.")

36 images found.


In [5]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image

    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)

    else:
        r = width / float(w)
        dim = (width, int(h * r))

    resized = cv2.resize(image, dim, interpolation = inter)

    return resized

def processed_filename(image_filename):
    filepath, filename_ext = os.path.split(image_filename)
    filename, file_ext = os.path.splitext(filename_ext)

    return filename, file_ext

In [6]:
n_images = 0
proc_imgs_dict = {}

for n_raw, image in enumerate(raw_image_lst):

    print(f"Processing image {n_raw + 1}/{len(raw_image_lst)}: {image.split('/',-1)[-1]} ...")
  
    # read image
    img = cv2.imread(image)
  
    # data augmentation 1: change size (416x416) 
    img1 = image_resize(img, height=IMAGE_SIZE)
    aug = CenterCrop(IMAGE_SIZE,IMAGE_SIZE, p=1.0)
    img1 = aug.apply(img1)

    aug_name = "resize"
    processed_image_filename, file_ext = processed_filename(image)
    proc_filename = f"{processed_images+processed_image_filename}_{aug_name}{file_ext}"
    cv2.imwrite(proc_filename, img1)
    proc_imgs_dict[aug_name] = img1
    n_images += 1

    # data augmentation 2: vertical mirroring
    aug = VerticalFlip(p=1)
    aug_name = "vertflip"
    img2 = aug.apply(img1)
    proc_filename = f"{processed_images+processed_image_filename}_{aug_name}{file_ext}" 
    cv2.imwrite(proc_filename, img2)
    proc_imgs_dict[aug_name] = img2
    n_images += 1

    # data augmentation 3: horizontal mirroring
    aug = HorizontalFlip(p=1)
    aug_name = "horzflip"
    img3 = aug.apply(img1)
    proc_filename = f"{processed_images+processed_image_filename}_{aug_name}{file_ext}"
    cv2.imwrite(proc_filename, img3)
    proc_imgs_dict[aug_name] = img3
    n_images += 1

    # data augmentation 4: elastic transformation
    aug = ElasticTransform(alpha=101, sigma=81, alpha_affine=53, p=0.5)
    aug_name = "elastic"
    img4 = aug.apply(img1)
    proc_filename = f"{processed_images+processed_image_filename}_{aug_name}{file_ext}"
    cv2.imwrite(proc_filename, img4)
    proc_imgs_dict[aug_name] = img4
    n_images += 1

    # data augmentation 5: rotation
    aug = RandomRotate90(p=1.0)
    aug_name = "rot"
    img5 = aug.apply(img1, factor=45)
    proc_filename = f"{processed_images+processed_image_filename}_{aug_name}{file_ext}"
    cv2.imwrite(proc_filename, img5)
    proc_imgs_dict[aug_name] = img5
    n_images += 1

    for proc_name in proc_imgs_dict:
    
        # data augmentation a: change alpha value
        alpha = 2.2
        aug = RandomContrast(limit=0.9, p=1.0)
        aug_name = "alpha"
        img6 = aug.apply(proc_imgs_dict[proc_name], alpha=alpha)
        proc_filename = f"{processed_images+processed_image_filename}_{proc_name}_{aug_name}{file_ext}"
        cv2.imwrite(proc_filename, img6)
        n_images += 1

        # data augmentation b: arbitrary brightness
        alpha = 1.2
        aug = RandomBrightness(limit=0.2, p=1.0)
        aug_name = "bright"
        img7 = aug.apply(proc_imgs_dict[proc_name], alpha=alpha)
        proc_filename = f"{processed_images+processed_image_filename}_{proc_name}_{aug_name}{file_ext}"
        cv2.imwrite(proc_filename, img7)
        n_images += 1
    
        # data augmentation c: gray value
        aug = ToGray(p=0.5)
        aug_name = "gray"
        img8 = aug.apply(proc_imgs_dict[proc_name], alpha=alpha)
        proc_filename = f"{processed_images+processed_image_filename}_{proc_name}_{aug_name}{file_ext}"
        cv2.imwrite(proc_filename, img8)
        n_images += 1

        # data augmentation d: sepia
        aug = ToSepia(p=1)
        aug_name = "sepia"
        img9 = aug.apply(proc_imgs_dict[proc_name], alpha=alpha)
        proc_filename = f"{processed_images+processed_image_filename}_{proc_name}_{aug_name}{file_ext}"
        cv2.imwrite(proc_filename, img9)
        n_images += 1

        # data augmentation e: blur
        aug = Blur(blur_limit=11, p=1)
        aug_name = "blur"
        img10 = aug.apply(proc_imgs_dict[proc_name], alpha=alpha)
        proc_filename = f"{processed_images+processed_image_filename}_{proc_name}_{aug_name}{file_ext}"
        cv2.imwrite(proc_filename, img10)
        n_images += 1

        # data augmentation f: noise
        aug = MultiplicativeNoise(always_apply=True, elementwise=True, multiplier=(0.9, 1.1), p=1.0)
        aug_name = "noise"
        img11 = aug.apply(proc_imgs_dict[proc_name], alpha=alpha)
        proc_filename = f"{processed_images+processed_image_filename}_{proc_name}_{aug_name}{file_ext}"
        cv2.imwrite(proc_filename, img11)
        n_images += 1

        # data augmentation g: JPEGCompression
        aug = JpegCompression(quality_lower=0, quality_upper=1, p=1)
        aug_name = "jpeg"
        img12 = aug.apply(proc_imgs_dict[proc_name], alpha=alpha)
        proc_filename = f"{processed_images+processed_image_filename}_{proc_name}_{aug_name}{file_ext}"
        cv2.imwrite(proc_filename, img12)
        n_images += 1

        # data augmentation h: color channel
        aug = ChannelDropout(channel_drop_range=(1, 1), fill_value=0, p=1)
        aug_name = "channel"
        img13 = aug.apply(proc_imgs_dict[proc_name])
        proc_filename = f"{processed_images+processed_image_filename}_{proc_name}_{aug_name}{file_ext}"
        cv2.imwrite(proc_filename, img13)
        n_images += 1

print("{} images generated.".format(n_images))


Processing image 1/36: 22.png ...
Processing image 2/36: 35.png ...
Processing image 3/36: 34.png ...
Processing image 4/36: 18.png ...
Processing image 5/36: 15.png ...
Processing image 6/36: 30.png ...
Processing image 7/36: 12.png ...
Processing image 8/36: 27.png ...
Processing image 9/36: 14.png ...
Processing image 10/36: 1.png ...
Processing image 11/36: 28.png ...
Processing image 12/36: 33.png ...
Processing image 13/36: 24.png ...
Processing image 14/36: 31.png ...
Processing image 15/36: 6.png ...
Processing image 16/36: 23.png ...
Processing image 17/36: 32.png ...
Processing image 18/36: 16.png ...
Processing image 19/36: 29.png ...
Processing image 20/36: 36.png ...
Processing image 21/36: 10.png ...
Processing image 22/36: 4.png ...
Processing image 23/36: 37.png ...
Processing image 24/36: 7.png ...
Processing image 25/36: 13.png ...
Processing image 26/36: 21.png ...
Processing image 27/36: 9.png ...
Processing image 28/36: 11.png ...
Processing image 29/36: 20.png ...